In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
async def get_tools():
    # 서버 설정 정의
    server_config = {
        "search": {
            "url": "http://localhost:8000/mcp",
            "transport": "sse",
            },
        }

    # 1. 'async with'를 제거하고 클라이언트를 직접 생성합니다.
    client = MultiServerMCPClient(server_config)
    
    # 2. 'await'를 사용하여 비동기적으로 도구를 가져옵니다.
    tools = await client.get_tools()
    return tools

tools = await get_tools()
tool_names = [tool.name for tool in tools]
print(tool_names)
tools

['wiki_search', 'safety_search', 'tech_paper_search']


[StructuredTool(name='wiki_search', description='Do Search\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json\n\n**Output Schema:**\n```json\n{}\n```\n**422**: Validation Error\nContent-Type: application/json\n\n**Example Response:**\n```json\n{\n  "detail": [\n    {\n      "loc": [],\n      "msg": "Message",\n      "type": "Error Type"\n    }\n  ]\n}\n```\n\n**Output Schema:**\n```json\n{\n  "properties": {\n    "detail": {\n      "items": {\n        "properties": {\n          "loc": {\n            "items": {},\n            "type": "array",\n            "title": "Location"\n          },\n          "msg": {\n            "type": "string",\n            "title": "Message"\n          },\n          "type": {\n            "type": "string",\n            "title": "Error Type"\n          }\n        },\n        "type": "object",\n        "required": [\n          "loc",\n          "msg",\n          "type"\n        ],\n        "title": "ValidationE

In [4]:
import nest_asyncio
import asyncio

nest_asyncio.apply()
from langchain_mcp_adapters.client import MultiServerMCPClient
# Add other necessary imports like ChatGroq, create_react_agent etc.

async def main(tools:list, query:str):

    # 서버 설정 정의
    server_config = {
        "search": {
            "url": "http://localhost:8000/mcp",
            "transport": "sse",
            },
        }

    # 1. 'async with'를 제거하고 클라이언트를 직접 생성합니다.
    client = MultiServerMCPClient(server_config)
    
    # 2. 'await'를 사용하여 비동기적으로 도구를 가져옵니다.
    tools = await client.get_tools()
    
    model = ChatGroq(
            model= "qwen/qwen3-32b", 
            temperature=0.5,
            max_tokens=1000,
            )
        
    prompt = """
        You are the Smart AI Assistant in a company.
        Based on the result of tool calling, Generate a consice and logical answer.
        and if there is no relevant infomation in the tool calling result, Just say 'I don't know'
        """
    
    agent = create_react_agent(model=model, tools=tools, prompt=prompt)
    
    inputs = {"messages": [("user", query)]}
    return await agent.ainvoke(inputs, stream_mode="values")

In [6]:
sel_tools = tools[0:2]
sel_tools

[StructuredTool(name='wiki_search', description='Do Search\n\n### Responses:\n\n**200**: Successful Response (Success Response)\nContent-Type: application/json\n\n**Output Schema:**\n```json\n{}\n```\n**422**: Validation Error\nContent-Type: application/json\n\n**Example Response:**\n```json\n{\n  "detail": [\n    {\n      "loc": [],\n      "msg": "Message",\n      "type": "Error Type"\n    }\n  ]\n}\n```\n\n**Output Schema:**\n```json\n{\n  "properties": {\n    "detail": {\n      "items": {\n        "properties": {\n          "loc": {\n            "items": {},\n            "type": "array",\n            "title": "Location"\n          },\n          "msg": {\n            "type": "string",\n            "title": "Message"\n          },\n          "type": {\n            "type": "string",\n            "title": "Error Type"\n          }\n        },\n        "type": "object",\n        "required": [\n          "loc",\n          "msg",\n          "type"\n        ],\n        "title": "ValidationE

In [12]:

# Example of how to run it (assuming the rest of the code is defined)
async def run_example():
    try:
        query = "knowledge distillation"
        result = await main(tools=sel_tools, query=query)
        return result
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

result = asyncio.run(run_example())
result

{'messages': [HumanMessage(content='knowledge distillation', additional_kwargs={}, response_metadata={}, id='7f0bc9d7-643b-4fbf-9952-115d4661db49'),
  AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user asked about "knowledge distillation." Let me think about how to approach this.\n\nFirst, I need to recall what knowledge distillation is. From what I remember, it\'s a technique in machine learning where a smaller model, called the student model, is trained to mimic the behavior of a larger, more complex model known as the teacher model. The idea is to transfer the knowledge from the teacher to the student so that the student can achieve similar performance but with lower computational costs.\n\nBut wait, maybe I should verify this. Let me consider using the wiki_search function to get a concise definition. The user might want a clear explanation without too much jargon. Alternatively, if the safety_search or tech_paper_search functions have relevant informatio

In [13]:
print(result["messages"][1].additional_kwargs["reasoning_content"])

Okay, the user asked about "knowledge distillation." Let me think about how to approach this.

First, I need to recall what knowledge distillation is. From what I remember, it's a technique in machine learning where a smaller model, called the student model, is trained to mimic the behavior of a larger, more complex model known as the teacher model. The idea is to transfer the knowledge from the teacher to the student so that the student can achieve similar performance but with lower computational costs.

But wait, maybe I should verify this. Let me consider using the wiki_search function to get a concise definition. The user might want a clear explanation without too much jargon. Alternatively, if the safety_search or tech_paper_search functions have relevant information, those could provide more detailed insights, especially if there are recent advancements or applications in technical papers.

Let me check the tools available. The wiki_search function takes a query parameter. If I u

In [14]:
result["messages"][1].additional_kwargs["tool_calls"][0]["function"]["name"]

'wiki_search'

In [15]:
result["messages"][2].content

'{\n  "results": [\n    {\n      "Page": "지식 증류",\n      "Summary": "지식 증류(知識蒸溜, knowledge distillation)는 지식을 증류한다는 뜻으로 서로 다른 기계학습 모델의 지식을 전하는 기법이다. 기계 학습에서 지식 증류 또는 모델 증류는 큰 모델에서 작은 모델로 지식을 전달하는 프로세스이다. 대규모 모델(예: 매우 심층적인 신경망 또는 여러 모델의 앙상블)은 소규모 모델보다 지식 용량이 높지만 이 용량이 완전히 활용되지 않을 수 있다. 모델이 지식 용량을 거의 활용하지 않더라도 모델을 평가하는 데 계산 비용이 많이 들 수 있다. 지식 증류는 유효성 손실 없이 큰 모델의 지식을 작은 모델로 이전한다. 소형 모델은 평가 비용이 저렴하므로 덜 강력한 하드웨어(예: 모바일 장치)에 배포할 수 있다. 지식 증류는 객체 감지, 음향 모델 및 자연어 처리와 같은 기계 학습의 여러 응용 분야에서 성공적으로 사용되었다. 최근에는 그리드가 아닌 데이터에 적용 가능한 그래프 신경망에도 도입되었다."\n    },\n    {\n      "Page": "증류",\n      "Summary": "증류(蒸溜)는 상대휘발도(끓는점)의 차이를 이용하여 액체 상태의 혼합물을 분리하는 방법이다. 두 혼합물의 화학 반응 없이 물리적인 분리가 이루어지는 경우를 말한다. 영리적인 목적의 증류는 석유를 증류하여 휘발유, 경유, 등유와 같은 여러 종류의 연료로 분리하는 데에서부터, 공기를 증류하여 아르곤과 같은 특수한 기체를 얻는 데 이르기까지 광범위하게 사용된다. 고체와 액체의 혼합물은 무게, 입자의 크기, 용해도의 차이 등을 이용하여 분리시킬 수 있지만 물과 에틸알코올처럼 액체와 액체의 혼합물을 분리시킬 때나 소금물에서 물만을 분리해내는 경우 등은 다른 방법이 필요하다. 물질은 각각 일정한 끓는점을 가지고 있다. 따라서, 끓는점의 차이가 있는 혼합물을 가열하면 끓는점이 낮은 것이 먼저 증기가 되어 나오므로 이것을 냉각시켜 

In [16]:
print(result["messages"][-1].content)

지식 증류(knowledge distillation)는 기계 학습에서 대규모 모델(예: 심층 신경망 또는 앙상블 모델)이 소유한 지식을 소형 모델로 이전하는 기법입니다. 이 과정을 통해 소형 모델은 큰 모델의 성능을 유지하면서도 계산 비용이 낮아져 모바일 장치 등 하드웨어 자원이 제한된 환경에서도 효율적으로 사용할 수 있습니다. 주요 응용 분야로는 객체 감지, 음향 모델, 자연어 처리 등이 있으며, 최근에는 그래프 신경망에도 확장되어 적용되고 있습니다.
